In [17]:
import transformers, tokenizers, datasets

In [18]:
from datasets import load_dataset

dataset = load_dataset('PolyAI/banking77')
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10003
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 3080
    })
})

In [19]:
from transformers import AutoTokenizer
model_id = 'bert-base-cased' 
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [20]:
print(dataset['train']['text'][0])
print(tokenizer.encode(dataset['train']['text'][0]))

I am still waiting on my card?
[101, 146, 1821, 1253, 2613, 1113, 1139, 3621, 136, 102]


In [21]:
tok_out = tokenizer(dataset['train']['text'][67], padding='max_length', max_length=15)
tokenizer.decode(tok_out['input_ids'])

'[CLS] how to track the card you sent [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]'

In [22]:
def tokenize(examples):
    return tokenizer(examples['text'], padding='max_length', max_length=128, truncation=True)

tokenizer_data = dataset.map(tokenize, batched=True)

Map: 100%|██████████| 3080/3080 [00:00<00:00, 7213.92 examples/s] 


In [23]:
labels = dataset['train'].features['label']
num_labels = len(labels.names)

In [24]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(28996, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [26]:
from transformers import Trainer, TrainingArguments
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer)

train_args = TrainingArguments(output_dir='')
trainer = Trainer( model=model,
    args=train_args,
    data_collator=data_collator,
    train_dataset=tokenizer_data['train'],
    eval_dataset=tokenizer_data['test']
)

trainer.train()

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [27]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
